# Adding a (physics-based) model

So far we've only looked at machine learning models. We are very keen to know
how these "smart" approaches compare to more traditional, physics-based models.

[PyPhenology](https://github.com/sdtaylor/pyPhenology) is a nice python package
with a collection of physics-based models. We would like to compare those
models, ideally within the same pycaret framework. However, pyPhenology is not
consistent with the scikit-learn API. On the other hand, it is quite possible to
cast their equations to a form that does adhere to these standards.

In this notebook, we will walk you through the steps to create a custom
estimator, following the [scikit-learn
documentation](https://scikit-learn.org/stable/developers/develop.html). We will
show how this is done for [pyPhenology's ThermalTime
model](https://pyphenology.readthedocs.io/en/master/generated/pyPhenology.models.ThermalTime.html#pyPhenology.models.ThermalTime).
At the end of the chapter, you should be able to repeat the trick for the other
pyPhenology models as well.

## The first blow is half the battle

As a starting point, we copied the [scikit-learn project template](https://github.com/scikit-learn-contrib/project-template/blob/a06bc1a701fbb320848e4d5295e4477b596078df/skltemplate/_template.py) and updated it with some of the information from the [pyphenology ThermalTime](https://github.com/sdtaylor/pyPhenology/blob/d82af2f669364e84be4bf9325a4f4e064d8d3816/pyPhenology/models/thermaltime.py) class. Specifically, we:

- Added
  [RegressorMixin](https://scikit-learn.org/stable/modules/generated/sklearn.base.RegressorMixin.html#sklearn.base.RegressorMixin)
  from scikit-learn. This contains some methods specific to regression estimators.
- Replaced `check_X_y` and `check_array` with the newer `_validate_data()` (see
  [SLEP010](https://scikit-learn-enhancement-proposals.readthedocs.io/en/latest/slep010/proposal.html)).
- Merged docstrings of ThermalTime and sklearn template
- Changed to google-style docstrings and added type hints to method signatures instead of in the docstrings
- Set the default values of the parameters to sensible ints instead of valid ranges


In [1]:
import numpy as np
from sklearn.base import (
    BaseEstimator,
    RegressorMixin,
    check_is_fitted,
)
from numpy.typing import ArrayLike


class ThermalTime(RegressorMixin, BaseEstimator):
    """Thermal Time Model

    The classic growing degree day model using a fixed temperature threshold
    above which forcing accumulates.
    """

    def __init__(self):
        pass

    # TODO: consider adding DOY index series to fit/predict as optional argument
    def fit(self, X: ArrayLike, y: ArrayLike):
        """Fit the model to the available observations.

        Parameters:
            X: 2D Array of shape (n_samples, n_features).
                Daily mean temperatures for each unique site/year (n_samples) and
                for each DOY (n_features). The first feature should correspond to
                the first DOY, and so forth up to (max) 366.
            y: 1D Array of length n_samples
                Observed DOY of the spring onset for each unique site/year.

        Returns:
            Fitted model
        """
        X, y = self._validate_data(X, y)
        # TODO: check additional assumptions about input

        # TODO: convert to proper fit; for now set some default values
        self.t1_: int = 0
        self.T_: int = 5
        self.F_: int = 500

        # `fit` should always return `self`
        return self

    def predict(self, X: ArrayLike):
        """Predict values of y given new predictors

        Parameters:
            X: array-like, shape (n_samples, n_features).
               Daily mean temperatures for each unique site/year (n_samples) and
               for each DOY (n_features). The first feature should correspond to
               the first DOY, and so forth up to (max) 366.

        Returns:
            y: array-like, shape (n_samples,)
               Predicted DOY of the spring onset for each sample in X.
        """
        X = self._validate_data(X)
        check_is_fitted(self, ["t1_", "T_", "F_"])

        # TODO: Implement real predictions
        return np.ones(X.shape[0], dtype=np.int64)

Phew! That's a big start! Notice that we're not passing in anything during initialization (yet). By convention of scikit-learn, the parameters of the model are only set during fit. Fitted parameters can be recognized by their trailing underscore.

This class can already be used, although it doesn't
actually fit or predict anything (useful) yet. Next, we need to

- Provide an implementation for the predict method
- Provide an implementation for the fit method
- Consider doing more validation of the input, since now we simply assume that
  the data is in ordere columns from DOY 1 up to (max) 366.
- Consider allowing an additional argument to fit and predict that contains the
  column indices in case they're not neatly formatted from 1 to (max) 366. This
  is allowed by scikit-learn as long as it's an optional argument.

However, before we proceed, let's see whether we already adhere to the
scikit-learn API.

## Checking sklearn compliance

Scikit-learn provide a nice compliance checker. With a bit of extra code we can
print out which tests it fails (see below).


In [2]:
from sklearn.utils.estimator_checks import check_estimator

# This bit of code allows us to run the checks in a notebook
checks = check_estimator(ThermalTime(), generate_only=True)
passed_checks = 0
failed_checks = 0
for estimator, check in checks:
    name = check.func.__name__
    try:
        check(estimator)
        passed_checks += 1
    except Exception as exc:
        print(f"Check {name} failed with exception: {exc}")
        failed_checks += 1
print(f"Passed checks: {passed_checks}, failed checks: {failed_checks}")

Check check_regressors_train failed with exception: 
Check check_regressors_train failed with exception: 
Check check_regressors_train failed with exception: 
Passed checks: 37, failed checks: 3


So far, so good. Most of the checks passed, and if we dive deep into what's
being check, we can figure out that the others failed because the predictions
were not that good. That makes sense...

## Using pytest and source files

While it is possible to do all this in a notebook, a neater and more convenient
way is to use pytest. To this end:

- Store the class definition above in a new file called `thermaltime.py`
- Create a new file called `test_thermaltime.py` and add the following content

  ```py
  from thermaltime import ThermalTime

  from sklearn.utils.estimator_checks import parametrize_with_checks

  @parametrize_with_checks([ThermalTime(),])
  def test_sklearn_compatible_estimator(estimator, check):
      check(estimator)
  ```

- Install pytest: `pip install pytest`
- Run pytest: `pytest test_thermaltime.py`

## Implementing predict

We'll start by implementing the predict method. This is relatively
straightforward. We'll write a simple function that takes both X and the
parameters, and returns the expected DOY. For ease of reference, we copied the
docstrings from above. This implementation should be exactly the same as in
pyPhenology.


In [3]:
# Note: Copy this function to your file thermaltime.py


def thermaltime(X, t1: int = 1, T: int = 5, F: int = 500):
    """Make prediction with the thermaltime model.

    X: array-like, shape (n_samples, n_features).
       Daily mean temperatures for each unique site/year (n_samples) and for
       each DOY (n_features). The first feature should correspond to
       the first DOY, and so forth up to (max) 366.
    t1: The DOY at which forcing accumulating beings (should be within [-67,298])
    T: The threshold above which forcing accumulates (should be within [-25,25])
    F: The total forcing units required (should be within [0,1000])
    """
    # This allows us to pass both 1D and 2D arrays of temperature
    # Copying X to safely modify it later on (may not be necessary, but readable)
    X_2d = np.atleast_2d(np.copy(X))

    # DOY starts at 1, where python array index start at 0
    # TODO: Make this an optional argument?
    doy = np.arange(X_2d.shape[1]) + 1

    # Exclude days before the start of the growing season
    X_2d[:, doy < t1] = 0

    # Exclude days with temperature below threshold
    X_2d[X_2d < T] = 0

    # Accumulate remaining data
    S = np.cumsum(X_2d, axis=-1)

    # Find first entry that exceeds the total forcing units required.
    doy = np.argmax(S > F, axis=-1)

    return doy

Let's see how we can use this model:


In [4]:
# 10 degrees every day:
X_test = np.ones(365) * 10

# Predicted spring onset:
thermaltime(X_test)

array([50])

In [5]:
# Also check for 2D X inputs:
X_test = np.ones((10, 365)) * 10
thermaltime(X_test)

array([50, 50, 50, 50, 50, 50, 50, 50, 50, 50])

Good, it seems this works nicely, both for indivual prediction and for 2D arrays of inputs.

### Adding tests

These quick checks are super useful! We can quickly add a few more and add them to our test file (`test_thermaltime.py`). Note: also copy the `thermaltime` function from above to your file `thermaltime.py`.


In [6]:
# Note: Copy these tests to your file thermaltime.py, uncomment the imports, and remove the bottom part.

# Note: these imports must be uncommented in your test file
# import numpy as np
# from thermaltime import ThermalTime, thermaltime


def test_1d_base_case():
    # 10 degrees every day:
    X_test = np.ones(365) * 10
    assert thermaltime(X_test) == 50


def test_late_growing_season():
    # If the growing season starts later, the spring onset is later as well.
    X_test = np.ones(365) * 10
    assert thermaltime(X_test, t1=11) == 60


def test_higher_threshold():
    # If the total accumulated forcing required is higher, spring onset is later.
    X_test = np.ones(365) * 10
    assert thermaltime(X_test, F=600) == 60


def test_exclude_cold_days():
    # If some days are below the minimum growing T, spring onset is later.
    X_test = np.ones(365) * 10
    X_test[[1, 4, 8, 12, 17, 24, 29, 33, 38, 42]] = 3
    assert thermaltime(X_test) == 60


def test_lower_temperature_threshold():
    # If the minimum growing T is lower, fewer days are exluded. However, the
    # accumulated temperature rises more slowly.
    X_test = np.ones(365) * 10

    X_test[[1, 4, 8, 12, 17, 24, 29, 33, 38, 42]] = 5
    assert thermaltime(X_test, T=2) == 55


def test_2d():
    # Should be able to predict for multiple samples at once
    X_test = np.ones((10, 365)) * 10
    expected = np.ones(10) * 50
    result = thermaltime(X_test)
    assert np.all(result == expected)


# Note: The following lines are not needed in your test file. Pytest will
# automatically call all functions starting with "test_".
test_1d_base_case()
test_late_growing_season()
test_higher_threshold()
test_exclude_cold_days()
test_lower_temperature_threshold()
test_2d()

After you've copied the code to your files, you can run pytest again to check that all new tests pass.

Now that we're confident our new predict function works, the last thing we need to do is update the predict method on the class. Change it to look like this:

```py
    def predict(self, X: ArrayLike):
        """Predict values of y given new predictors

        Parameters:
            X: array-like, shape (n_samples, n_features).
               Daily mean temperatures for each unique site/year (n_samples) and
               for each DOY (n_features). The first feature should correspond to
               the first DOY, and so forth up to (max) 366.

        Returns:
            y: array-like, shape (n_samples,)
               Predicted DOY of the spring onset for each sample in X.
        """
        X = self._validate_data(X)
        check_is_fitted(self, ["t1_", "T_", "F_"])

        return thermaltime(X, self.t1_, self.T_, self.F_)
```


## Implementing the `fit` method

Now that we can make predictions, we can also think about optimizing the parameters of the model.

### Generate training data

Our aim is to minimize the difference between the predictions based on the training data `X` and the target data `y`. Let's first prepare some training data to test the method once we have it.


In [7]:
# Prepare some training data. Take a base temperature of 10 degrees and add a
# random fluctuation on top of it. The corresponding spring onset should
# correlate with the random temperature fluctations.

temp_signal = np.random.randn(10, 365)
X_train = np.ones((10, 365)) * 10 + temp_signal * 10
y_train = np.ones(10) * 50 + temp_signal.mean(axis=1) * 100

# Check that the values are within somewhat realistic ranges
print(X_train.min(), X_train.max())
print(y_train.min(), y_train.max())

-27.44049847799723 46.040299933994405
40.856081333695 58.4199093709297


### Exploring algorithms

Scipy has a lot of [optimization routines](https://docs.scipy.org/doc/scipy/reference/optimize.html). Looking at the list of optimizers, it seems `curve-fit` has the exact signature we are looking for. However, if we try it, we will quickly find out that it is not fit for purpose.


In [8]:
from scipy.optimize import curve_fit

initial_guess = [1, 5, 500]
lower_bounds = [-67, -25, 0]
upper_bounds = [298, 25, 1000]

curve_fit(
    thermaltime, X_train, y_train, p0=initial_guess, bounds=(lower_bounds, upper_bounds)
)

(array([  1.00000005,   5.        , 500.        ]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]))

This gives terrible fits! Or rather, it doesn't seem to do much fitting at all. If you play around with the initial settings you'll see that most of the output is rubbish. That's probably because our prediction model is not a nice, continuous functional form. Also, `curve-fit` [doesn't really like](https://stackoverflow.com/a/22861933) integer parameters such as the DOY (though our implementation doesn't really care either).

Clearly, we need to look further.

### Reformulating the problem

Alternatively, following pyphenology, we can try the [global optimizers](https://docs.scipy.org/doc/scipy/reference/optimize.html) instead. Looking at the documentation of the [brute-force optimizer](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brute.html#scipy.optimize.brute), it seems we need to reformulate our problem a little bit. First, we need to define a loss function, i.e. the function that scipy can minimize.

Intuitively, our loss function could look something like this:


In [9]:
def rmse_loss(X, y, t1: int = 1, T: int = 5, F: int = 500):
    y_pred = thermaltime(X, t1=t1, T=T, F=F)
    sq_err = (y_pred - y) ** 2
    return np.mean(sq_err) ** 0.5


# Try it with the default parameters:
rmse_loss(X_train, y_train)

12.2831808222873

However, this is not consistent with the form expected by scipy's global optimizers: they want something of the form `rmse_loss(x, *args)`. So we need to reformulate our loss function in terms of `x` and `args`.

Nota bene: where the brute-force example finds the minimum of the function in the x-y space given fixed parameters, we want to do the opposite: we want to find the minimum in the parameter space, given the (fixed for each training batch) X and y. So, `x` is a tuple of our models params, `*args` corresponds to `X` and `y`, and our loss function becomes something like this:


In [10]:
def rmse_loss(params, X, y):
    """RMSE loss between thermaltime predictions and observations.

    params is a tuple with the parameters to thermaltime: (t1, T, F)
    """
    y_pred = thermaltime(X, *params)
    sq_err = (y_pred - y) ** 2
    return np.mean(sq_err) ** 0.5


# Try it with the default parameters:
rmse_loss([1, 5, 500], X_train, y_train)

12.2831808222873

While the above is good enough, we can stil tweak our loss function a bit further while we're at it. Currently, our loss has the `thermaltime` function hardcoded. It would be much nicer if this was more flexible, so it could work for any (roughly similar) model, for example, all the pyphenology models.

One way to do this is to pass the predictor function as one of the arguments.


In [11]:
def rmse_loss(params, f, X, y):
    """RMSE loss between thermaltime predictions and observations.

    f is a prediction model of the form f(X, *params)
    params is a tuple with the parameters to f
    """
    y_pred = f(X, *params)
    sq_err = (y_pred - y) ** 2
    return np.mean(sq_err) ** 0.5


# Try it with the default parameters:
rmse_loss([1, 5, 500], thermaltime, X_train, y_train)

12.2831808222873

### Brute force optimization

Now we're ready to run the brute force optimization.


In [12]:
from scipy.optimize import brute

ranges = ((-67, 298), (-25, 25), (0, 1000))
args = (thermaltime, X_train, y_train)

brute(rmse_loss, ranges=ranges, args=args)

array([50.67631579, 14.47368421,  0.        ])

That works! If you look carefully at the results, you will see that the optimization algorithm finds a clever solution to the problem: it sets the total forcing required to 0, and the first day of the growing season very close to the typical spring onset. While this is a perfectly valid solution, especially considering the fake training data we used, it illustrates that you need to be really careful in setting the expected bounds of the parameters, and in interpreting the results.

Another thing to note is how scipy creates the search space. The `brute` function has an additional parameter `Ns` that determines the number of 'grid points' used for each of the parameters. The default is 20: the ranges we supplied for `t1`, `T`, and `F` are divided in 20 points, such that the algorithm executes our loss function `20 * 20 * 20` times. By increasing the number of points, we get a better estimate, but the number of function calls and therefore the training time increases exponentially.

There are two ways around this:

- Set the points yourself
- Use a more clever algorithm

The first solution is quite simple. We can also define the ranges as slices with a custom `step`:


In [13]:
ranges = (slice(-67, 298, 10), slice(-25, 25, 2), slice(0, 1000, 50))

brute(rmse_loss, ranges=ranges, args=args)

array([ 53., -25.,   0.])

### Basin hopping

For the second solution, again, we follow the pyphenology developers. They support "differential evolution" and "basin_hopping". Let's try the basin hopping algorithm. This algorithm works in two steps, delegating calls to our loss function to a local minimization function. Thus, we need to pass in our bounds and args as `minimzer_kwargs` instead. We must also pick a method for the local minizer. For consistency with pyphenology, we choose 'LL-BFGS-B'.


In [14]:
from scipy.optimize import basinhopping

initial_guess = (1, 5, 500)
minimizer_args = {
    "method": "L-BFGS-B",
    "args": (thermaltime, X_train, y_train),
    "bounds": ((-67, 298), (-25, 25), (0, 1000)),
}
basinhopping(rmse_loss, x0=initial_guess, minimizer_kwargs=minimizer_args)

                    message: ['requested number of basinhopping iterations completed successfully']
                    success: True
                        fun: 10.389960273765986
                          x: [ 2.244e+00  5.994e+00  4.984e+02]
                        nit: 100
      minimization_failures: 0
                       nfev: 472
                       njev: 118
 lowest_optimization_result:  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
                              success: True
                               status: 0
                                  fun: 10.389960273765986
                                    x: [ 2.244e+00  5.994e+00  4.984e+02]
                                  nit: 0
                                  jac: [ 0.000e+00  0.000e+00  0.000e+00]
                                 nfev: 4
                                 njev: 1
                             hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

This also works, but notice that the result is still very close to the initial
guess. If we change the initial guess to something that's more like the clever
solution of the brute method, we find another fit much smaller RMSE value:


In [15]:
initial_guess = (45, 15, 0)
minimizer_args = {
    "method": "L-BFGS-B",
    "args": (thermaltime, X_train, y_train),
    "bounds": ((-67, 298), (-25, 25), (0, 1000)),
}

bh = basinhopping(rmse_loss, x0=initial_guess, minimizer_kwargs=minimizer_args)
print(f"params: {bh.x}, minimum: {bh.fun}")

params: [47.04348765 15.83528259  2.65394513], minimum: 6.633890538987595


Notice that we can pass additional arguments to the basinhopping algorithm to tweak its performance. For example, the default of 100 iterations is quite small. Using the settings from pyphenology, we get a result that looks much more realistic (and takes much longer to train):


In [16]:
from scipy.optimize import basinhopping

initial_guess = (1, 5, 500)
minimizer_args = {
    "method": "L-BFGS-B",
    "args": (thermaltime, X_train, y_train),
    "bounds": ((-67, 298), (-25, 25), (0, 1000)),
}
optimizer_params = {"niter": 50000, "T": 0.5, "stepsize": 0.5, "disp": False}
basinhopping(
    rmse_loss, x0=initial_guess, **optimizer_params, minimizer_kwargs=minimizer_args
)

                    message: ['requested number of basinhopping iterations completed successfully']
                    success: True
                        fun: 5.3850952685237115
                          x: [ 5.078e+01 -7.646e+00  9.683e+00]
                        nit: 50000
      minimization_failures: 0
                       nfev: 200072
                       njev: 50018
 lowest_optimization_result:  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
                              success: True
                               status: 0
                                  fun: 5.3850952685237115
                                    x: [ 5.078e+01 -7.646e+00  9.683e+00]
                                  nit: 0
                                  jac: [ 0.000e+00  0.000e+00  0.000e+00]
                                 nfev: 4
                                 njev: 1
                             hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

Great! Now we have a good grip on the different ways to fit our model. A final thing to point out is that the basinhopping algorithm returns floats for each parameter. This doesn't really make sense for the start of the growing season, so in the post-processing we should probably round that to the nearest int.


## Bringing everything together

If we bring everything from above over to the `fit` method, we get something like this:

```python
    def fit(self, X: ArrayLike, y: ArrayLike):
        """Fit the model to the available observations.

        Parameters:
            X: 2D Array of shape (n_samples, n_features).
                Daily mean temperatures for each unique site/year (n_samples) and
                for each DOY (n_features). The first feature should correspond to
                the first DOY, and so forth up to (max) 366.
            y: 1D Array of length n_samples
                Observed DOY of the spring onset for each unique site/year.

        Returns:
            Fitted model
        """
        X, y = self._validate_data(X, y)
        # TODO: check additional assumptions about input

        initial_guess = (1, 5, 500)
        bounds = ((-67, 298), (-25, 25), (0, 1000))

        minimizer_args = {
            "method": "L-BFGS-B",
            "args": (thermaltime, X, y),
            "bounds": bounds,
        }
        optimizer_params = {"niter": 50000, "T": 0.5, "stepsize": 0.5, "disp": False}

        bh = basinhopping(
            partial(rmse_loss, f=thermaltime, X=X, y=y),
            x0=initial_guess,
            minimizer_kwargs=minimizer_args,
            **optimizer_params,
        )

        self.t1_, self.T_, self.F_ = bh.x

        return self
```

This is not yet ready:

- `Initial guess` and `bounds` are hardcoded and closely related to the equally hardcoded `thermaltime` and the loss function. If we want to reuse this for other physics-based models, it would be nice to try and separate these a bit better.
- The `optimizer_params` are hardcoded as well. These are typical (hyper) parameters that can be passed to the `__init__` method of our class, since these settings are not data-dependent.
- Currently the only optimization routine supported is the basinhopping algorithm.

The version below is slightly better. We generalized the loss function a bit better, and the core_model-related parameters are grouped together at the top. The fit method is mostly concerned with formulating the call to the basinhopping algorithm. We used the `staticmethod` decorator to make the thermaltime function a method on the class. Notice that, in contrast with standard class methods, it doesn't need access to `self`.


In [17]:
import numpy as np
from sklearn.base import (
    BaseEstimator,
    RegressorMixin,
    check_is_fitted,
)
from numpy.typing import ArrayLike
from scipy.optimize import basinhopping


def rmse_loss(params, f, X, y):
    """RMSE loss between thermaltime predictions and observations.

    f is a prediction model of the form f(X, *params)
    params is a tuple with the parameters to f
    """
    y_pred = f(X, *params)
    sq_err = (y_pred - y) ** 2
    return np.mean(sq_err) ** 0.5


LOSS_FUNCTIONS = {
    "RMSE": rmse_loss,
}


class ThermalTime(RegressorMixin, BaseEstimator):
    """Thermal Time Model

    The classic growing degree day model using a fixed temperature threshold
    above which forcing accumulates.

    This implementation uses scipy's basinhopper optimization algorithm for
    fitting the model.
    """

    _core_params_names = ("t1", "T", "F")
    _core_params_defaults = (1, 5, 500)
    _core_params_bounds = ((-67, 298), (-25, 25), (0, 1000))

    def __init__(
        self,
        niter=50000,
        T=0.5,
        stepsize=0.5,
        disp=False,
        minimizer_method="L-BFGS-B",
        loss_function="RMSE",
    ):
        self.niter = niter
        self.T = T
        self.stepsize = stepsize
        self.disp = disp
        self.minimizer_method = minimizer_method
        self.loss_function = loss_function

    @staticmethod
    def _core_model(X, t1: int = 1, T: int = 5, F: int = 500):
        """Make prediction with the thermaltime model.

        Args:
            X: array-like, shape (n_samples, n_features).
                Daily mean temperatures for each unique site/year (n_samples) and for
                each DOY (n_features). The first feature should correspond to
                the first DOY, and so forth up to (max) 366.
            t1: The DOY at which forcing accumulating beings (should be within [-67,298])
            T: The threshold above which forcing accumulates (should be within [-25,25])
            F: The total forcing units required (should be within [0,1000])
        """
        # This allows us to pass both 1D and 2D arrays of temperature
        # Copying X to safely modify it later on (may not be necessary, but readable)
        X_2d = np.atleast_2d(np.copy(X))

        # DOY starts at 1, where python array index start at 0
        # TODO: Make this an optional argument?
        doy = np.arange(X_2d.shape[1]) + 1

        # Exclude days before the start of the growing season
        X_2d[:, doy < t1] = 0

        # Exclude days with temperature below threshold
        X_2d[X_2d < T] = 0

        # Accumulate remaining data
        S = np.cumsum(X_2d, axis=-1)

        # Find first entry that exceeds the total forcing units required.
        doy = np.argmax(S > F, axis=-1)

        return doy

    # TODO: consider adding DOY index series to fit/predict as optional argument
    def fit(self, X: ArrayLike, y: ArrayLike):
        """Fit the model to the available observations.

        Parameters:
            X: 2D Array of shape (n_samples, n_features).
                Daily mean temperatures for each unique site/year (n_samples) and
                for each DOY (n_features). The first feature should correspond to
                the first DOY, and so forth up to (max) 366.
            y: 1D Array of length n_samples
                Observed DOY of the spring onset for each unique site/year.

        Returns:
            Fitted model
        """
        X, y = self._validate_data(X, y)
        # TODO: check additional assumptions about input

        loss_function = LOSS_FUNCTIONS[self.loss_function]

        # Perform the fit
        bh = basinhopping(
            loss_function,
            x0=self._core_params_defaults,
            niter=self.niter,
            T=self.T,
            stepsize=self.stepsize,
            disp=self.disp,
            minimizer_kwargs={
                "method": self.minimizer_method,
                "args": (self._core_model, X, y),
                "bounds": self._core_params_bounds,
            },
        )

        # Store the fitted parameters
        self.core_params_ = bh.x

        return self

    def predict(self, X: ArrayLike):
        """Predict values of y given new predictors

        Parameters:
            X: array-like, shape (n_samples, n_features).
               Daily mean temperatures for each unique site/year (n_samples) and
               for each DOY (n_features). The first feature should correspond to
               the first DOY, and so forth up to (max) 366.

        Returns:
            y: array-like, shape (n_samples,)
               Predicted DOY of the spring onset for each sample in X.
        """
        X = self._validate_data(X)
        check_is_fitted(self, "core_params_")

        return self._core_model(X, *self.core_params_)

In [ ]:
Update tests:

* redefine thermaltime
* set n_iter to something more sensible